> Please go through Giba's post and kernel  to underrstand what this leak is all about
> https://www.kaggle.com/titericz/the-property-by-giba (kernel)
> https://www.kaggle.com/c/santander-value-prediction-challenge/discussion/61329 (post)
> 
> Also, go through this Jiazhen's kernel which finds more columns to exploit leak
> https://www.kaggle.com/johnfarrell/giba-s-property-extended-result
> 
> I just exploit data property in brute force way and then fill in remaining by row non zero means! This should bring everyone on level-playing field.
> 
> **Let the competition begin! :D**

### Just some small modifications from [original baseline](https://www.kaggle.com/tezdhar/breaking-lb-fresh-start)~
- The leak rows are calculated separately on train/test set
- Calculated the leaky values, correctness, for each lag
- Hope this can help to do some *lag_selection*

### Update leak process codes to Dmitry Frumkin's *fast* [version](https://www.kaggle.com/dfrumkin/a-simple-way-to-use-giba-s-features-v2)
- The result of Dmitry's original function and result of Hasan's function seem slightly different
- Modified to make the output consistent with Hasan's function (Seems better score)

In [1]:
import warnings
warnings.filterwarnings('ignore')
import gc

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# print(os.listdir("../input"))

import lightgbm as lgb
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error, make_scorer
from scipy.stats import mode, skew, kurtosis, entropy
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import dask.dataframe as dd
from dask.multiprocessing import get

from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)

# Any results you write to the current directory are saved as output.

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

transact_cols = [f for f in train.columns if f not in ["ID", "target"]]
y = np.log1p(train["target"]).values

In [3]:
test["target"] = train["target"].mean()

# Determine Extra Features to use

In [4]:
rows = [1757, 3809, 511, 3798, 625, 3303, 4095, 1283, 4209, 1696, 3511, 816, 245, 1383, 2071, 3492, 378, 2971, 2366, 4414, 2790, 3979, 193, 1189, 3516, 810, 4443, 3697, 235, 1382, 4384, 3418, 4396, 921, 3176, 650]

def element_wise_comparison(i1, i2, thresh=3):
    i1 = np.array(i1)
    i2 = np.array(i2)
    dif = i1 - i2
    count = np.count_nonzero(dif)
    if count <= thresh:
        return True
    else:
        return False

def solve_ties_in_nb_of_fillers(df, counts, stop_thresh=2):
    args = np.argsort(counts)[::-1]
    stop = np.squeeze(np.where(counts[args] == len(df.index)-stop_thresh))
#     print(stop, type(stop))
    try:
        stop = int(stop)
    except:
          return [None]
#     print(args)
    u, c = np.unique(counts[args[stop:]], return_counts=True)
    conflicts = u[np.where(c > 1)]
#     print(conflicts)
    for c in conflicts:
#         print(c)
        idx = np.squeeze(np.where(counts == c))
        ref_id = np.squeeze(np.where(counts == c+1))
#         print('ref_id', ref_id)
        ref = df.iloc[:-1, int(ref_id)]
#         print(ref)
#         print(df.iloc[1:, idx[0]].values - ref.values)
#         print(df.iloc[1:, idx[1]].values - ref.values)
        dif1 = np.count_nonzero(df.iloc[1:, idx[0]].values - ref.values)
        dif2 = np.count_nonzero(df.iloc[1:, idx[1]].values - ref.values)
#         print('difs', dif1, dif2)
#         print('idx_value0', df.iloc[:, idx[0]], df.iloc[:, idx[1]])
        loc0  = np.where(args == idx[0])
        loc1  = np.where(args == idx[1])
#         print('Locs (index do args)', loc0, loc1)
        tmp0 = args[loc0]
        tmp1 = args[loc1]
#         print('tmp (values do args)', tmp0, tmp1)
#         print(np.minimum(loc0, loc1), np.maximum(loc0, loc1))
        if dif2 < dif1:
            args[np.minimum(loc0, loc1)] = tmp1
            args[np.maximum(loc0, loc1)] = tmp0
        else:
            args[np.minimum(loc0, loc1)] = tmp0
            args[np.maximum(loc0, loc1)] = tmp1           
#         print('tmp depois(values do args)', tmp0, tmp1)

#             print('Switch')
#             print('Idx (index do count)', idx[0], idx[1])
        

#             args[loc0], args[loc1] = args[loc1], args[loc0] 

#     print(args)
    return args[stop:]
    
        
def find_filler_value(df, idx):
    u, c = np.unique(df.iloc[0,:].values, return_counts=True)
    arg = np.argsort(c)
    return u[arg][-idx]

def find_col_order_by_filler(df, filler, stop_thresh=2, nb=40):
    counts = []
    for c in df.columns:
        v = df[c].values - filler
        counts.append(np.count_nonzero(v))
    counts = np.array(counts)
    args = solve_ties_in_nb_of_fillers(df, counts, stop_thresh)
    if not all(args):
        return [None]
    return df.iloc[:, args]

def find_leading_col(raw_df, sorted_df):
    curr_lead = sorted_df.apply(tuple, axis=0).to_frame().rename(columns={0: 'key'}).iloc[0, 0][1:]
    cols_to_match = raw_df.apply(tuple, axis=0).to_frame().rename(columns={0: 'key'})
    for c in cols_to_match.index:
        if element_wise_comparison(cols_to_match.loc[c, 'key'][:-1], curr_lead):
            print('Leading col found: {}'.format(c))
            return pd.concat([raw_df[c], sorted_df], axis=1)
        
    print('No lead found')
    return sorted_df
    
def find_n_cols_groups(df, n_groups=40, stop_thresh=2):
    col_groups = []
    for i in range(n_groups):
        filler = find_filler_value(tmp, i)
        df_view = find_col_order_by_filler(tmp, filler)
        if not all(df_view):
            continue
        for _ in range(stop_thresh + 1):
            df_view = find_leading_col(tmp, df_view)
        col_groups.append(list(df_view.columns))
    return col_groups

In [47]:
cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 
        'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 'd6bb78916', 'b43a7cfd5', 
        '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 
        'fb49e4212', '66ace2992', 'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', 
        '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', 
        '6619d81fc', '1db387535', 
        'fc99f9426',
        '91f701ba2', '0572565c2', '190db8488', 'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98'
       ]
    
extra_feats = ['ced6a7e91','9df4daa99','83c3779bf','edc84139a','f1e0ada11','73687e512',
               'aa164b93b','342e7eb03','cd24eae8a','8f3740670','2b2a10857','a00adf70e',
               '3a48a2cd2','a396ceeb9','9280f3d04','fec5eaf1a','5b943716b','22ed6dba3',
               '5547d6e11','e222309b0','5d3b81ef8','1184df5c2','2288333b4','f39074b55',
               'a8b721722','13ee58af1','fb387ea33','4da206d28','ea4046b8d','ef30f6be5',
               'b85fa8b27','2155f5e16','794e93ca6','070f95c99','939f628a7','7e814a30d',
               'a6e871369','0dc4d6c7d','bc70cbc26','aca228668']

# extra_feats = [[]]
tmp = train.loc[rows, :].drop('ID', axis=1)
extra_feats = find_n_cols_groups(tmp, 85)
extra_feats

Leading col found: 4e92107c6
Leading col found: 53aa182a2
No lead found
Leading col found: 18562fc62
Leading col found: 81de0d45e
No lead found
Leading col found: d8e951a15
Leading col found: 3391c0af1
No lead found
Leading col found: 9126049d8
Leading col found: a3e023f65
No lead found
Leading col found: ca7ea80a3
Leading col found: ccc7609f4
No lead found
Leading col found: 2b8851e90
Leading col found: 48b839509
No lead found
Leading col found: 34d3715d5
Leading col found: d0d340214
No lead found
Leading col found: 93efdb50f
Leading col found: 920a04ee2
No lead found
Leading col found: c30ff7f31
Leading col found: 0f8d7b98e
No lead found
Leading col found: 9d4428628
No lead found
No lead found
Leading col found: b850c3e18
Leading col found: 5030aed26
No lead found
Leading col found: 9df4daa99
Leading col found: ced6a7e91
No lead found
Leading col found: 36a56d23e
Leading col found: 63be1f619
No lead found
Leading col found: 63c094ba4
Leading col found: 87ffda550
No lead found
Leading

[['53aa182a2',
  '4e92107c6',
  '295408598',
  'b76bf3f19',
  '3305c8063',
  'd3a116347',
  'ac5260727',
  '199caef5d',
  '97ea72529',
  '1d4d5cd4a',
  '8fc7efaf0',
  '225fa9d61',
  '94f3dcaee',
  '4634c8fae',
  '660fdbc58',
  '052f633c1',
  '657dec16b',
  '7fa5bc19f',
  '7207afb67',
  'cda277b2a',
  'e9a473fbb',
  '3eac9a76e',
  '1c554649c',
  '86ffb104c',
  'b14d5014b',
  '8348ea8d3',
  'e3a4596f9',
  '49db469f5',
  'f928893ca',
  'aa610feec',
  'fa2a340da',
  '652142369',
  '947c7c3e8',
  'f81908ca5',
  '8160230fd',
  'c2d200f0e',
  'c99902a93',
  'd3a6362c5',
  '3ee95e3ef',
  '7f8027faf'],
 ['81de0d45e',
  '18562fc62',
  '543c24e33',
  '0256b6714',
  'd6006ff44',
  '6a323434b',
  'e3a38370e',
  '7c444370b',
  '8d2d050a2',
  '9657e51e1',
  '13f3a3d19',
  'b5c839236',
  '70f3033c6',
  'f4b374613',
  '849125d91',
  '16b532cdc',
  '88219c257',
  '74fb8f14c',
  'fd1102929',
  '699712087',
  '22501b58e',
  '9e9274b24',
  '2c42b0dce',
  '2c95e6e31',
  '5263c204d',
  '526ed2bec',
  '01f7de

In [6]:
len(extra_feats)

39

In [38]:

cols_2 = ['ced6a7e91','9df4daa99','83c3779bf','edc84139a','f1e0ada11','73687e512',
                'aa164b93b','342e7eb03','cd24eae8a','8f3740670','2b2a10857','a00adf70e',
                '3a48a2cd2','a396ceeb9','9280f3d04','fec5eaf1a','5b943716b','22ed6dba3',
                '5547d6e11','e222309b0','5d3b81ef8','1184df5c2','2288333b4','f39074b55',
                'a8b721722','13ee58af1','fb387ea33','4da206d28','ea4046b8d','ef30f6be5',
                'b85fa8b27','2155f5e16']
cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 
        'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 'd6bb78916', 'b43a7cfd5', 
        '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 
        'fb49e4212', '66ace2992', 'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', 
        '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', 
        '6619d81fc', '1db387535', 
        'fc99f9426',
        '91f701ba2', '0572565c2', '190db8488', 'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98'
       ]
# x = extra_feats

In [7]:
# x = extra_feats
extra_feats = extra_feats[5:]
# del extra_feats[11]

In [36]:
extra_feats

[['ced6a7e91',
  '9df4daa99',
  '83c3779bf',
  'edc84139a',
  'f1e0ada11',
  '73687e512',
  'aa164b93b',
  '342e7eb03',
  'cd24eae8a',
  '8f3740670',
  '2b2a10857',
  'a00adf70e',
  '3a48a2cd2',
  'a396ceeb9',
  '9280f3d04',
  'fec5eaf1a',
  '5b943716b',
  '22ed6dba3',
  '5547d6e11',
  'e222309b0',
  '5d3b81ef8',
  '1184df5c2',
  '2288333b4',
  'f39074b55',
  'a8b721722',
  '13ee58af1',
  'fb387ea33',
  '4da206d28',
  'ea4046b8d',
  'ef30f6be5',
  'b85fa8b27',
  '2155f5e16',
  '794e93ca6',
  '070f95c99',
  '939f628a7',
  '7e814a30d',
  'a6e871369',
  '0dc4d6c7d',
  'bc70cbc26',
  'aca228668']]

In [168]:
extra_feats[13]
extra_feats = [extra_feats[6]]
# extra_feats = [extra_feats[11]]

In [79]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
for i, c in enumerate(extra_feats):
    if not i:
        x = pd.concat([train.loc[rows, ['target']+cols], train.loc[rows, c]], axis=1)
    else:
        x = pd.concat([x, train.loc[rows, c]], axis=1)
# display(train.loc[rows, extra_feats[25]])
display(x)

,target,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,20aa07010,024c577b9,d6bb78916,b43a7cfd5,58232a6fb,1702b5bf0,324921c7b,62e59a501,2ec5b290f,241f0f867,fb49e4212,66ace2992,f74e8f13d,5c6487af1,963a49cdc,26fc93eb7,1931ccfdd,703885424,70feb1494,491b9ee45,23310aa6f,e176a204a,6619d81fc,1db387535,fc99f9426,91f701ba2,0572565c2,190db8488,adb64ff71,c47340d97,c5a231d81,0ff32eb98,3391c0af1,d8e951a15,320c3880a,37bd241bb,ef6fab1bc,39d765b49,c0f2800fc,9401c82a9,6f9d256a9,f967f827b,798a8e92e,1e991f09b,77854931e,0024cd760,ab7b12083,7b8ac394f,d8aa77bcc,54b3d6afc,4c6a1accd,0a7ef2f9b,dc3b4460b,83ce5d80d,7e293fbaf,8d468092c,dc0118d47,9f8df01bd,3f02111a8,bf0195c11,28bc37378,05410a84f,ecdc88e4b,8d9263a24,380d54fad,e234cdc9b,7f17e0b2a,03e726456,8f66157e3,556fb6bc0,2d2114d9b,215ffb087,a3e023f65,9126049d8,6eaea198c,5244415dd,0616154cc,2165c4b94,fc436be29,1834f29f5,9d5af277d,c6850e7db,6b241d083,56f619761,45319105a,fcda960ae,07746dcda,c906cd268,c24ea6548,829fb34b8,89ebc1b76,22c019a2e,1e16f11f3,94072d7a3,59dfc16da,9886b4d22,0b1741a7f,a682ef110,e26299c3a,5c220a143,ac0493670,8d8bffbae,68c7cf320,3cea34020,e9a8d043d,afb6b8217,5780e6ffa,26628e8d8,1de4d7d62,4c53b206e,99cc87fd7,593cccdab,ccc7609f4,ca7ea80a3,e509be270,3b8114ab0,a355497ac,27998d0f4,fa05fd36e,81aafdb57,4e22de94f,f0d5ffe06,9af753e9d,f1b6cc03f,567d2715c,857020d0f,99fe351ec,3e5dab1e3,001476ffa,5a5eabaa7,cb5587baa,32cab3140,313237030,0f6386200,b961b0d59,9452f2c5f,bcfb439ee,04a22f489,7e58426a4,a4c9ea341,ffdc4bcf8,1a6d866d7,d7334935b,298db341e,08984f627,8367dfc36,5d9f43278,7e3e026f8,37c10d610,5a88b7f01,324e49f36,99f466457,48b839509,2b8851e90,28f75e1a5,0e3ef9e8f,37ac53919,7ca10e94b,4b6c549b1,467aa29ce,74c5d55dc,0700acbe1,44f3640e4,e431708ff,097836097,d1fd0b9c2,a0453715a,9e3aea49a,899dbe405,525635722,87a2d8324,faf024fa9,d421e03fd,1254b628a,a19b05919,34a4338bc,08e89cc54,a29c9f491,a0a8005ca,62ea662e7,5fe6867a4,8b710e161,7ab926448,d04e16aed,4e5da0e96,ff2c9aa8f,b625fe55a,7124d86d9,215c4d496,b6fa5a5fd,55a7e0643,0a26a3cfe,87ffda550,63c094ba4,2e103d632,1c71183bb,d5fa73ead,e078302ef,a6b6bc34a,f6eba969e,0d51722ca,ce3d7595b,6c5c8869c,dfd179071,122c135ed,b4cfe861f,b7c931383,44d5b820f,4bcf15776,51d4053c7,1fe5d56b9,ea772e115,ad009c8b9,68a945b18,62fb56487,c10f31664,cbb673163,c8d582dd2,8781e4b91,bd6da0cca,ca2b906e8,11e12dbe8,bb0ce54e9,c0d2348b7,77deffdf0,f97d9431e,a09a238d0,935ca66a9,9de83dc23,861076e21,f02ecb19c,166008929,920a04ee2,93efdb50f,15ea45005,78c57d7cd,91570fb11,c5dacc85b,145c7b018,590b24ab1,c283d4609,e8bd579ae,7298ca1ef,ce53d1a35,a8f80f111,2a9fed806,feb40ad9f,cfd255ee3,31015eaab,303572ae2,cd15bb515,cb5161856,a65b73c87,71d64e3f7,ec5fb550f,4af2493b6,18b4fa3f5,3d655b0ed,5cc9b6615,88c0ec0a6,8722f33bb,5ed0c24d0,54f26ee08,04ecdcbb3,ade8a5a19,d5efae759,ac7a97382,e1b20c3a6,b0fcfeab8,438b8b599,43782ef36,df69cf626,9d4428628,37f11de5d,39549da61,ceba761ec,4c60b70b8,304ebcdbc,823ac378c,4e21c4881,5ee81cb6e,eb4a20186,f6bdb908a,6654ce6d8,65aa7f194,00f844fea,c4de134af,a240f6da7,168c50797,13d6a844f,7acae7ae9,8c61bede6,45293f374,feeb05b3f,a5c62af4a,22abeffb6,1d0aaa90f,c46028c0f,337b3e53b,d6af4ee1a,cde3e280a,c83fc48f2,f99a09543,85ef8a837,a31ba11e6,64cabb6e7,93521d470,46c525541,cef9ab060,375c6080e,3c4df440f,e613715cc,5030aed26,b850c3e18,212efda42,9e7c6b515,2d065b147,49ca7ff2e,37c85a274,ea5ed6ff7,deabe0f4c,bae4f747c,ca96df1db,05b0f3e9a,eb19e8d63,235b8beac,85fe78c6c,cc507de6c,e0bb9cf0b,80b14398e,9ca0eee11,4933f2e67,fe33df1c4,e03733f56,1d00f511a,e62cdafcf,3aad48cda,d36ded502,92b13ebba,f30ee55dd,1f8754c4e,db043a30f,e75cfcc64,5d8a55e6d,6e29e9500,c5aa7c575,c2cabb902,d251ee3b4,73700eaa4,8ab6f5695,54b1c1bc0,cbd0256fb,81de0d45e,18562fc62,543c24e33,0256b6714,d6006ff44,6a323434b,e3a38370e,7c444370b,8d2d050a2,9657e51e1,13f3a3d19,b5c839236,70f3033c6,f4b374613,849125d91,16b532cdc,88219c257,74fb8f14c,fd1102929,699712087,22501b58e,9e9274b24,2c42b0dce,2c95e6e31,5263c204d,526ed2bec,01f7de15d,cdbe394fb,adf357c9b,d0f65188c,b8a716ebf,ef1e1fac8,a3f2345bf,110e4132e,586b23138,680159bab,f1a1562cd,9f2f1099b,bf0e69e55,af

# Find leaks

In [34]:
# from: https://www.kaggle.com/dfrumkin/a-simple-way-to-use-giba-s-features-v2
def _get_leak(df, cols, extra_feats=[[]], lag=0):
    f1 = cols[:-lag-2]
    f2 = cols[lag+2:]
    for ef in extra_feats:
        f1 += ef[:-lag-2]
        f2 += ef[lag+2:]
    
    
    d1 = df[f1].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2 = df[f2].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2['pred'] = df[cols[lag]]
    #d2 = d2[d2.pred != 0] ### to make output consistent with Hasan's function
    d3 = d2[~d2.duplicated(['key'], keep=False)]
    return d1.merge(d3, how='left', on='key').pred.fillna(0)

In [43]:
def compiled_leak_result(extra_cols):    
    max_nlags = len(cols) - 2
    train_leak = train
    train_leak["compiled_leak"] = 0
    train_leak["nonzero_mean"] = train[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        
        print('Processing lag', i)
        train_leak[c] = _get_leak(train_leak, cols, extra_cols, i)
        
        leaky_cols.append(c)
#         train_leak = train.join(train_leak[leaky_cols+["compiled_leak", "nonzero_mean", "ID"]], on='ID', how='left')
        zeroleak = train_leak["compiled_leak"]==0
        train_leak.loc[zeroleak, "compiled_leak"] = train_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(train_leak["compiled_leak"] > 0))
        _correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        print("Leak values found in train", leaky_value_counts[-1])
        print(
            "% of correct leaks values in train ", 
            leaky_value_corrects[-1]
        )
        tmp = train_leak.copy()
        tmp.loc[zeroleak, "compiled_leak"] = train_leak["nonzero_mean"]
        scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        print(
            'Score (filled with nonzero mean)', 
            scores[-1]
        )
        gc.collect()
    result = dict(
        score=scores, 
        leaky_count=leaky_value_counts,
        leaky_correct=leaky_value_corrects,
    )
    return train_leak, result

# Find helpful cols

In [48]:
good_cols = []
_, result = compiled_leak_result([[]])
ref = min(result['score'])
for i in range(len(extra_feats)):
    extra_cols = [extra_feats[i]]
    _, result = compiled_leak_result(extra_cols)
    if min(result['score']) < ref:
        print('Good col found, leader is ', extra_feats[i][0])
        good_cols.append(extra_feats[i])


Processing lag 0
Leak values found in train 1351
% of correct leaks values in train  0.9955588453
Score (filled with nonzero mean) 1.51383333916
Processing lag 1
Leak values found in train 1947
% of correct leaks values in train  0.996404725218
Score (filled with nonzero mean) 1.29220481295
Processing lag 2
Leak values found in train 2340
% of correct leaks values in train  0.99358974359
Score (filled with nonzero mean) 1.17328290468
Processing lag 3
Leak values found in train 2586
% of correct leaks values in train  0.993039443155
Score (filled with nonzero mean) 1.08432637367
Processing lag 4
Leak values found in train 2754
% of correct leaks values in train  0.993464052288
Score (filled with nonzero mean) 1.032787044
Processing lag 5
Leak values found in train 2899
% of correct leaks values in train  0.993101069334
Score (filled with nonzero mean) 0.99403242995
Processing lag 6
Leak values found in train 3014
% of correct leaks values in train  0.992368944924
Score (filled with nonz

% of correct leaks values in train  0.989746510966
Score (filled with nonzero mean) 0.78894768159
Processing lag 18
Leak values found in train 3527
% of correct leaks values in train  0.989509498157
Score (filled with nonzero mean) 0.781231267034
Processing lag 19
Leak values found in train 3542
% of correct leaks values in train  0.989553924337
Score (filled with nonzero mean) 0.776301984368
Processing lag 20
Leak values found in train 3554
% of correct leaks values in train  0.989589195273
Score (filled with nonzero mean) 0.770517106787
Processing lag 21
Leak values found in train 3570
% of correct leaks values in train  0.988795518207
Score (filled with nonzero mean) 0.761591456256
Processing lag 22
Leak values found in train 3580
% of correct leaks values in train  0.988547486034
Score (filled with nonzero mean) 0.754038852777
Processing lag 23
Leak values found in train 3593
% of correct leaks values in train  0.988310603952
Score (filled with nonzero mean) 0.751137975895
Processi

% of correct leaks values in train  0.983783783784
Score (filled with nonzero mean) 0.722150236606
Processing lag 35
Leak values found in train 3701
% of correct leaks values in train  0.983517968117
Score (filled with nonzero mean) 0.723748207881
Processing lag 36
Leak values found in train 3706
% of correct leaks values in train  0.982191041554
Score (filled with nonzero mean) 0.726659488018
Processing lag 37
Leak values found in train 3707
% of correct leaks values in train  0.981926085784
Score (filled with nonzero mean) 0.727110760968
Good col found, leader is  81de0d45e
Processing lag 0
Leak values found in train 1293
% of correct leaks values in train  0.99535962877
Score (filled with nonzero mean) 1.51383333916
Processing lag 1
Leak values found in train 1868
% of correct leaks values in train  0.99625267666
Score (filled with nonzero mean) 1.30563207373
Processing lag 2
Leak values found in train 2262
% of correct leaks values in train  0.993368700265
Score (filled with nonzer

% of correct leaks values in train  0.993306169965
Score (filled with nonzero mean) 0.790533335868
Processing lag 14
Leak values found in train 3471
% of correct leaks values in train  0.993373667531
Score (filled with nonzero mean) 0.776078208639
Processing lag 15
Leak values found in train 3491
% of correct leaks values in train  0.993411629905
Score (filled with nonzero mean) 0.7634977163
Processing lag 16
Leak values found in train 3508
% of correct leaks values in train  0.993158494869
Score (filled with nonzero mean) 0.758755110311
Processing lag 17
Leak values found in train 3530
% of correct leaks values in train  0.993201133144
Score (filled with nonzero mean) 0.755068063268
Processing lag 18
Leak values found in train 3546
% of correct leaks values in train  0.993231810491
Score (filled with nonzero mean) 0.747752733197
Processing lag 19
Leak values found in train 3563
% of correct leaks values in train  0.993264103284
Score (filled with nonzero mean) 0.742792848931
Processin

Leak values found in train 3673
% of correct leaks values in train  0.985842635448
Score (filled with nonzero mean) 0.717581546956
Processing lag 31
Leak values found in train 3677
% of correct leaks values in train  0.98504215393
Score (filled with nonzero mean) 0.718630472196
Processing lag 32
Leak values found in train 3680
% of correct leaks values in train  0.984239130435
Score (filled with nonzero mean) 0.718807413584
Processing lag 33
Leak values found in train 3692
% of correct leaks values in train  0.981852654388
Score (filled with nonzero mean) 0.719737197047
Processing lag 34
Leak values found in train 3693
% of correct leaks values in train  0.981857568373
Score (filled with nonzero mean) 0.718641025309
Processing lag 35
Leak values found in train 3693
% of correct leaks values in train  0.981857568373
Score (filled with nonzero mean) 0.718637512077
Processing lag 36
Leak values found in train 3698
% of correct leaks values in train  0.980530016225
Score (filled with nonze

...

Score (filled with nonzero mean) 0.935452665847
Processing lag 7
Leak values found in train 3137
% of correct leaks values in train  0.993624481989
Score (filled with nonzero mean) 0.890085294889
Processing lag 8
Leak values found in train 3217
% of correct leaks values in train  0.993472179049
Score (filled with nonzero mean) 0.863620340189
Processing lag 9
Leak values found in train 3268
% of correct leaks values in train  0.992962056304
Score (filled with nonzero mean) 0.845950431476
Processing lag 10
Leak values found in train 3326
% of correct leaks values in train  0.99308478653
Score (filled with nonzero mean) 0.832070033521
Processing lag 11
Leak values found in train 3367
% of correct leaks values in train  0.993168993169
Score (filled with nonzero mean) 0.817765521151
Processing lag 12
Leak values found in train 3416
% of correct leaks values in train  0.991803278689
Score (filled with nonzero mean) 0.801519797122
Processing lag 13
Leak values found in train 3449
% of correct

% of correct leaks values in train  0.987657948869
Score (filled with nonzero mean) 0.803725881188
Processing lag 24
Leak values found in train 3413
% of correct leaks values in train  0.986815118664
Score (filled with nonzero mean) 0.798594777214
Processing lag 25
Leak values found in train 3419
% of correct leaks values in train  0.9868382568
Score (filled with nonzero mean) 0.798265570818
Processing lag 26
Leak values found in train 3428
% of correct leaks values in train  0.985997666278
Score (filled with nonzero mean) 0.794399199143
Processing lag 27
Leak values found in train 3433
% of correct leaks values in train  0.985726769589
Score (filled with nonzero mean) 0.79299322376
Processing lag 28
Leak values found in train 3439
% of correct leaks values in train  0.985170107589
Score (filled with nonzero mean) 0.791018617032
Processing lag 29
Leak values found in train 3443
% of correct leaks values in train  0.984316003485
Score (filled with nonzero mean) 0.790400737421
Processing

Leak values found in train 2347
% of correct leaks values in train  0.995739241585
Score (filled with nonzero mean) 1.17004390868
Processing lag 3
Leak values found in train 2595
% of correct leaks values in train  0.994990366089
Score (filled with nonzero mean) 1.07887979886
Processing lag 4
Leak values found in train 2763
% of correct leaks values in train  0.995294969236
Score (filled with nonzero mean) 1.02701522587
Processing lag 5
Leak values found in train 2911
% of correct leaks values in train  0.99484713157
Score (filled with nonzero mean) 0.988034244196
Processing lag 6
Leak values found in train 3027
% of correct leaks values in train  0.994383878427
Score (filled with nonzero mean) 0.941131901119
Processing lag 7
Leak values found in train 3122
% of correct leaks values in train  0.994234465086
Score (filled with nonzero mean) 0.896593673734
Processing lag 8
Leak values found in train 3200
% of correct leaks values in train  0.9940625
Score (filled with nonzero mean) 0.870

Leak values found in train 3567
% of correct leaks values in train  0.990468180544
Score (filled with nonzero mean) 0.758782916006
Processing lag 20
Leak values found in train 3579
% of correct leaks values in train  0.990500139704
Score (filled with nonzero mean) 0.752792299293
Processing lag 21
Leak values found in train 3596
% of correct leaks values in train  0.989988876529
Score (filled with nonzero mean) 0.743653938513
Processing lag 22
Leak values found in train 3609
% of correct leaks values in train  0.989470767526
Score (filled with nonzero mean) 0.73614687984
Processing lag 23
Leak values found in train 3623
% of correct leaks values in train  0.989235440243
Score (filled with nonzero mean) 0.7325389564
Processing lag 24
Leak values found in train 3633
% of correct leaks values in train  0.988439306358
Score (filled with nonzero mean) 0.727145640428
Processing lag 25
Leak values found in train 3639
% of correct leaks values in train  0.988458367683
Score (filled with nonzero

Processing lag 36
Leak values found in train 3728
% of correct leaks values in train  0.985515021459
Score (filled with nonzero mean) 0.691305222886
Processing lag 37
Leak values found in train 3732
% of correct leaks values in train  0.984458735263
Score (filled with nonzero mean) 0.695794106247
Good col found, leader is  df838756c
Processing lag 0
Leak values found in train 1356
% of correct leaks values in train  0.997050147493
Score (filled with nonzero mean) 1.51383333916
Processing lag 1
Leak values found in train 1956
% of correct leaks values in train  0.997443762781
Score (filled with nonzero mean) 1.28974229953
Processing lag 2
Leak values found in train 2347
% of correct leaks values in train  0.995313165744
Score (filled with nonzero mean) 1.16955833739
Processing lag 3
Leak values found in train 2589
% of correct leaks values in train  0.995751255311
Score (filled with nonzero mean) 1.0793458686
Processing lag 4
Leak values found in train 2758
% of correct leaks values in 

In [49]:
train_leak, result = compiled_leak_result(good_cols)

Processing lag 0
Leak values found in train 1388
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 1.51383333916
Processing lag 1
Leak values found in train 2010
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 1.2798076585
Processing lag 2
Leak values found in train 2407
% of correct leaks values in train  0.999584545077
Score (filled with nonzero mean) 1.14727168219
Processing lag 3
Leak values found in train 2667
% of correct leaks values in train  0.999625046869
Score (filled with nonzero mean) 1.04762867686
Processing lag 4
Leak values found in train 2840
% of correct leaks values in train  0.999647887324
Score (filled with nonzero mean) 0.989106633692
Processing lag 5
Leak values found in train 2993
% of correct leaks values in train  0.99966588707
Score (filled with nonzero mean) 0.948694049968
Processing lag 6
Leak values found in train 3115
% of correct leaks values in train  0.999678972713
Score (filled with nonzero mean) 0.89949

In [ ]:
result = pd.DataFrame.from_dict(result, orient='columns')
result.T

In [ ]:
result.to_csv('train_leaky_stat.csv', index=False)

In [ ]:
train_leak.head()

In [ ]:
best_score = np.min(result['score'])
best_lag = np.argmin(result['score'])
print('best_score', best_score, '\nbest_lag', best_lag)

In [ ]:
def rewrite_compiled_leak(leak_df, lag):
    leak_df["compiled_leak"] = 0
    for i in range(lag):
        c = "leaked_target_"+str(i)
        zeroleak = leak_df["compiled_leak"]==0
        leak_df.loc[zeroleak, "compiled_leak"] = leak_df.loc[zeroleak, c]
    return leak_df

In [ ]:
leaky_cols = [c for c in train_leak.columns if 'leaked_target_' in c]
train_leak = rewrite_compiled_leak(train_leak, best_lag)
train_leak[['ID']+leaky_cols+['compiled_leak']].head()

In [ ]:
train_res = train_leak[leaky_cols+['compiled_leak']].replace(0.0, np.nan)
train_res.to_csv('train_leak.csv', index=False)

In [ ]:
def compiled_leak_result_test(max_nlags):
    test_leak = test[["ID", "target"] + cols]
    test_leak["compiled_leak"] = 0
    test_leak["nonzero_mean"] = test[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    # leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        
        print('Processing lag', i)
        test_leak[c] = _get_leak(test_leak, cols, i)
        
        leaky_cols.append(c)
        test_leak = test.join(
            test_leak.set_index("ID")[leaky_cols+["compiled_leak", "nonzero_mean"]], 
            on="ID", how="left"
        )[["ID", "target"] + cols + leaky_cols+["compiled_leak", "nonzero_mean"]]
        zeroleak = test_leak["compiled_leak"]==0
        test_leak.loc[zeroleak, "compiled_leak"] = test_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(test_leak["compiled_leak"] > 0))
        #_correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        #leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        print("Leak values found in test", leaky_value_counts[-1])
        #print(
        #    "% of correct leaks values in train ", 
        #    leaky_value_corrects[-1]
        #)
        #tmp = train_leak.copy()
        #tmp.loc[zeroleak, "compiled_leak"] = tmp.loc[zeroleak, "nonzero_mean"]
        #scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        #print(
        #    'Score (filled with nonzero mean)', 
        #    scores[-1]
        #)
    result = dict(
        # score=scores, 
        leaky_count=leaky_value_counts,
        # leaky_correct=leaky_value_corrects,
    )
    return test_leak, result

In [ ]:
test_leak, test_result = compiled_leak_result_test(max_nlags=38)

In [ ]:
test_result = pd.DataFrame.from_dict(test_result, orient='columns')
test_result.T

In [ ]:
test_result.to_csv('test_leaky_stat.csv', index=False)

In [ ]:
test_leak = rewrite_compiled_leak(test_leak, best_lag)
test_leak[['ID']+leaky_cols+['compiled_leak']].head()

In [ ]:
test_res = test_leak[leaky_cols+['compiled_leak']].replace(0.0, np.nan)
test_res.to_csv('test_leak.csv', index=False)

In [ ]:
# test_leak.loc[test_leak["compiled_leak"]==0, "compiled_leak"] = test_leak.loc[test_leak["compiled_leak"]==0, "nonzero_mean"]
test_leak.loc[test_leak["compiled_leak"]==0, "compiled_leak"] = 0

In [ ]:
#submission
sub = test[["ID"]]
sub["target"] = test_leak["compiled_leak"]
sub.to_csv(f"baseline_sub_lag_{best_lag}.csv", index=False)
print(f"baseline_sub_lag_{best_lag}.csv saved")

In [ ]:
sub.head()